## Implementation of Merge Sort in Julia

In [1]:
using BenchmarkTools

In [2]:
]status BenchmarkTools

Status `~/.julia/environments/v1.5/Project.toml`
  [6e4b80f9] BenchmarkTools v0.5.0


In [3]:
function merge1(A::Array, p::Integer, q::Integer, r::Integer)
  L = A[p:q]
  push!(L, Inf)
  R = A[q+1:r]
  push!(R, Inf)
  i = 1
  j = 1
  for k in p:r
    if L[i] <= R[j]
      A[k] = L[i]
      i += 1
    else
      A[k] = R[j]
      j += 1
    end
  end
end

merge1 (generic function with 1 method)

**(?)** Here, which would be better? Using `view` or using copy? When implementing a `MERGE` procedure w/o sentinel, try implement one using `view`.<br>
**(R)** My opinion is that we cannot use `view`. Cf. The section on **Performance Measure** below.

In [4]:
# Don't use push!
function merge2(A::Array, p::Integer, q::Integer, r::Integer)
  n1 = q - p + 1
  n2 = r - q
  L = zeros(n1+1)
  R = zeros(n2+1)
  L[1:end-1] = A[p:q]
  L[end] = Inf
  R[1:end-1] = A[q+1:r]
  R[end] = Inf
  i = 1
  j = 1
  for k in p:r
    if L[i] <= R[j]
      A[k] = L[i]
      i += 1
    else
      A[k] = R[j]
      j += 1
    end
  end
end

merge2 (generic function with 1 method)

In [5]:
typeof(Function)

DataType

In [6]:
merge1 isa Function

true

In [7]:
# Int or other type is better?
#function merge_sort(A::Array, p::Integer, r::Integer; merge::Function=merge2)
function merge_sort(A::Array, p::Integer, r::Integer)
  if p < r
    q = floor((p+r)/2)
    merge_sort(A, p, q)
    merge_sort(A, q+1, r)
    merge2(A, p, q, r)
  end
end

merge_sort (generic function with 1 method)

In [8]:
Int == Int64

true

In [9]:
Int <: Integer

true

In [10]:
Base.show_supertypes(Int)

Int64 <: Signed <: Integer <: Real <: Number <: Any

**(?)** Actually, the input args `p, q, r` above will always be positive integers (because they represents indices). Wouldn't specifying/restricting
the type declaration in the function argument to `::Unsigned` a better choice?

In [11]:
A = rand(1:100, 10)

10-element Array{Int64,1}:
 34
 55
 86
  8
 26
 35
 68
 42
 48
  7

In [12]:
merge_sort(A, 1, length(A))  # in-place sort
A

LoadError: MethodError: no method matching merge_sort(::Array{Int64,1}, ::Int64, ::Float64)
Closest candidates are:
  merge_sort(::Array, ::Integer, !Matched::Integer) at In[7]:3

In [13]:
typeof(length(A))

Int64

In [14]:
merge_sort(A, 1, 10)

LoadError: MethodError: no method matching merge_sort(::Array{Int64,1}, ::Int64, ::Float64)
Closest candidates are:
  merge_sort(::Array, ::Integer, !Matched::Integer) at In[7]:3

In [15]:
methods(merge_sort)

# 1 method for generic function "merge_sort":
[1] merge_sort(A::Array, p::Integer, r::Integer) in Main at In[7]:3

**(?)** Why the second arg is recognized as a `Float`?<br>
**(R)** After some confusion, I finally realize the cause:
- It's **not** the calls like `merge_sort(A, 1, length(A))` or `merge_sort(A, 1, 10)` which cause problems
- It's rather **the implementation of** `merge_sort` **itself** which causes problems

Indeed, the `q = floor((p+r)/2)` is suspicious. The return value of the function `floor` is probably by default `Float64`.

In [16]:
floor(3.14)

3.0

In [17]:
typeof(floor(3.14))

Float64

Let's correct our def of `merge_sort`. The doc says that we can restrict the return value's type by `floor(T, x)`.<br>
(cf. [https://docs.julialang.org/en/v1/base/math/#Base.floor](https://docs.julialang.org/en/v1/base/math/#Base.floor))

Moreover, let's add to our new function a **keyword argument with default value** `merge::Function=merge2`, giving it the capability to switch its `MERGE` procedure.

However, before that, let's **erase** the old and wrong `merge_sort`.

## Delete A Method: [`Base.delete_method`](https://discourse.julialang.org/t/deleting-methods-in-julia/13455)
```julia
julia> f(x::Integer) = 3.14           
f (generic function with 1 method)

julia> f(x::Int) = 2.71
f (generic function with 2 methods)

julia> subtypes(Integer)
3-element Array{Any,1}:
 Bool
 Signed
 Unsigned

julia> subtypes(Bool)
Type[]

julia> subtypes(Unsigned)
5-element Array{Any,1}:
 UInt128
 UInt16
 UInt32
 UInt64
 UInt8

julia> f(1)
2.71

julia> f(Integer(1))
2.71

julia> f(Unsigned(1))
3.14

julia> m = @which f(1)
f(x::Int64) in Main at REPL[2]:1

julia> m
f(x::Int64) in Main at REPL[2]:1

julia> methods(f)
# 2 methods for generic function "f":
[1] f(x::Int64) in Main at REPL[2]:1
[2] f(x::Integer) in Main at REPL[1]:1

julia> Base.delete_method(m)

julia> methods(f)
# 1 method for generic function "f":
[1] f(x::Integer) in Main at REPL[1]:1
```

In [18]:
?Base.delete_method

```
delete_method(m::Method)
```

Make method `m` uncallable and force recompilation of any methods that use(d) it.


In [19]:
methods(merge_sort)

# 1 method for generic function "merge_sort":
[1] merge_sort(A::Array, p::Integer, r::Integer) in Main at In[7]:3

In [20]:
?Base.delete_method

```
delete_method(m::Method)
```

Make method `m` uncallable and force recompilation of any methods that use(d) it.


In [21]:
Base.delete_method(merge_sort)

LoadError: MethodError: no method matching delete_method(::typeof(merge_sort))
Closest candidates are:
  delete_method(!Matched::Method) at reflection.jl:1326

**(?3)** `::Method` and `::Function`.

In [22]:
Base.show_supertypes(Method)

Method <: Any

In [23]:
Base.show_supertypes(Function)

Function <: Any

In [24]:
m = @which merge_sort(A, 1, length(A))

merge_sort(A::Array, p::Integer, r::Integer) in Main at In[7]:3

In [25]:
Base.delete_method(m)

In [26]:
methods(merge_sort)

# 0 methods for generic function "merge_sort":

## New, Corrected Definition of `merge_sort`

In [27]:
function merge_sort(A::Array, p::Int, r::Int; merge::Function=merge2)
  if p < r
    q = floor(Integer, (p+r)/2)
    merge_sort(A, p, q)
    merge_sort(A, q+1, r)
    merge(A, p, q, r)
  end
end

merge_sort (generic function with 1 method)

In [28]:
methods(merge_sort)

# 1 method for generic function "merge_sort":
[1] merge_sort(A::Array, p::Int64, r::Int64; merge) in Main at In[27]:1

In [29]:
merge_sort(A, 1, length(A))  # in-place sort
A

10-element Array{Int64,1}:
  7
  8
 26
 34
 35
 42
 48
 55
 68
 86

The reason for which I would like to at least run `merge_sort` once is because the procedure in the book (including its Julia implementation)
looks incredibly and weirdly simple, just a single `while` loop with three lines inside. I can sort of convince myself of its correctness but verifying whether it sorts correctly programmatically gives some more assurance/comfort.

Besides, note that the code works
- not only for arrays of length power of $2$
- but also arrays of arbitrary length

## Performance Measure
Let's define some more `MERGE` procedures.

In [30]:
# Don't use the ∞ sentinel
function merge3(A::Array, p::Integer, q::Integer, r::Integer)
  n1 = q - p + 1
  n2 = r - q
  L = A[p:q]
  R = A[q+1:r]
  i = 1
  j = 1
  for k in p:r
    # Must check here whether one of L and R is empty, e.g. whether i or j is out of index range
    if i > n1
      A[k:r] = R[j:end]
      break
    end
    if j > n2
      A[k:r] = L[i:end]
      break
    end
    if L[i] <= R[j]
      A[k] = L[i]
      i += 1
    else
      A[k] = R[j]
      j += 1
    end
  end
end

merge3 (generic function with 1 method)

Let's check the correctness of `merge3`

In [34]:
A = rand(1:100, 10)
println("(Before) A = $A")
merge_sort(A, 1, length(A); merge=merge3)
println("(After) A = $A")

(Before) A = [86, 45, 68, 1, 14, 22, 48, 97, 64, 43]
(After) A = [1, 14, 22, 43, 45, 48, 64, 68, 86, 97]


In [36]:
size(A)

(10,)

**(?)** At present `size(A)` equals `(n,)`. Can your `MERGE` and `MERGE-SORT` implementation handle the case when `size(A)` equals `(n,1)` or `(1,n)`?<br>
**(R)** It can already deal with arrays of those shapes/sizes.

In [77]:
A = reshape(rand(10), (1,10))
println("(Before) A = $A")
println("size(A) = $(size(A))")
println()
merge_sort(A, 1, length(A); merge=merge3)
println("(After) A = $A")
println("size(A) = $(size(A))")

(Before) A = [0.502507829641994 0.7700718682247021 0.548349325678082 0.22911090580810245 0.03038002191298439 0.5979759208294226 0.6292036518961777 0.06704665845272628 0.803108926573846 0.7841476522820399]
size(A) = (1, 10)

(After) A = [0.03038002191298439 0.06704665845272628 0.22911090580810245 0.502507829641994 0.548349325678082 0.5979759208294226 0.6292036518961777 0.7700718682247021 0.7841476522820399 0.803108926573846]
size(A) = (1, 10)


In [78]:
A = reshape(rand(10), (10,1))
println("(Before) A = $A")
println("size(A) = $(size(A))")
println()
merge_sort(A, 1, length(A); merge=merge3)
println("(After) A = $A")
println("size(A) = $(size(A))")

(Before) A = [0.5219402669824906; 0.6535290964304388; 0.45576798181125544; 0.8345284215675444; 0.8913706365078133; 0.8595168570704448; 0.3300831619529432; 0.9053434055393417; 0.41617106633873124; 0.8033845132052739]
size(A) = (10, 1)

(After) A = [0.3300831619529432; 0.41617106633873124; 0.45576798181125544; 0.5219402669824906; 0.6535290964304388; 0.8033845132052739; 0.8345284215675444; 0.8595168570704448; 0.8913706365078133; 0.9053434055393417]
size(A) = (10, 1)


The reason is that the same indexing and the same slicing in the code works for all these shapes/sizes of arrays:

In [38]:
# Don't use the ∞ sentinel and use view()
function merge4(A::Array, p::Integer, q::Integer, r::Integer)
  n1 = q - p + 1
  n2 = r - q
  L = view(A, p:q)
  R = @view A[q+1:r]  # equiv. to view(A, q+1 ,r)
  i = 1
  j = 1
  for k in p:r
    # Must check here whether one of L and R is empty, e.g. whether i or j is out of index range
    if i > n1
      A[k:r] = R[j:end]
      break
    end
    if j > n2
      A[k:r] = L[i:end]
      break
    end
    if L[i] <= R[j]
      A[k] = L[i]
      i += 1
    else
      A[k] = R[j]
      j += 1
    end
  end
end

merge4 (generic function with 1 method)

Let's test `merge4` -- I don't think it will work.

In [39]:
A = rand(1:100, 10)
println("(Before) A = $A")
merge_sort(A, 1, length(A); merge=merge4)
println("(After) A = $A")

(Before) A = [50, 26, 89, 34, 61, 37, 26, 38, 88, 11]
(After) A = [11, 11, 11, 11, 11, 11, 26, 37, 38, 88]


Reason? See the following example.

```julia
julia> A = [10, 7]
2-element Array{Int64,1}:
 10
  7

julia> L = @view A[1]
0-dimensional view(::Array{Int64,1}, 1) with eltype Int64:
10

julia> R = @view A[2]
0-dimensional view(::Array{Int64,1}, 2) with eltype Int64:
7

julia> A[1] = R[1]
7

julia> L
0-dimensional view(::Array{Int64,1}, 1) with eltype Int64:
7

julia> R
0-dimensional view(::Array{Int64,1}, 2) with eltype Int64:
7

julia> A
2-element Array{Int64,1}:
 7
 7
```

> So, we should not use `view` when implementing `MERGE-SORT`.

In [48]:
function test_efficiency(merge::Function)
  #A = rand(100)
  A = Array(100:1)
  merge_sort(A, 1, length(A); merge=merge)
end

test_efficiency (generic function with 1 method)

**(?)** Try to make `merge_sort` able to accept `A` as `100:1`, instead of having to convert to `Array(100:1)`.

In [68]:
@benchmark test_efficiency(merge1)

BenchmarkTools.Trial: 
  memory estimate:  80 bytes
  allocs estimate:  1
  --------------
  minimum time:     35.622 ns (0.00% GC)
  median time:      38.737 ns (0.00% GC)
  mean time:        50.354 ns (11.31% GC)
  maximum time:     4.621 μs (98.26% GC)
  --------------
  samples:          10000
  evals/sample:     992

In [69]:
@benchmark test_efficiency(merge2)

BenchmarkTools.Trial: 
  memory estimate:  80 bytes
  allocs estimate:  1
  --------------
  minimum time:     35.821 ns (0.00% GC)
  median time:      40.481 ns (0.00% GC)
  mean time:        54.783 ns (11.58% GC)
  maximum time:     5.489 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     992

In [70]:
@benchmark test_efficiency(merge3)

BenchmarkTools.Trial: 
  memory estimate:  80 bytes
  allocs estimate:  1
  --------------
  minimum time:     35.869 ns (0.00% GC)
  median time:      38.543 ns (0.00% GC)
  mean time:        47.121 ns (11.72% GC)
  maximum time:     3.913 μs (98.51% GC)
  --------------
  samples:          10000
  evals/sample:     993

**(?)** So, which `merge` implementation is better than which? I ask so, because, if we run the above cells (The `@benchmark` ones) multiple times, we'd get different results. Although more often it seems that `merge1 > merge2 > merge3` but at times we get `merge3 > merge1 > merge2`.<br>
**(R)** I guess their performances are more or less the same, the best probably is `merge1`.<br>

**(?)** But then, can you explain why `merge1` performs best?<br>
**(R)** I know not the answer, but if I must try to answer the question, maybe it's because although `merge3` and `merge1`'s worst case run time are both $\Theta(n)$, due to the fact that, at each iteration of `for k in p:r`, the code of `merge3` must check whether or not `L` or `R` is exhausted, that increases the constant running time of each iteration step.